### OPEN API 를 이용한 챗봇 시스템 

In [3]:
!pip show openapi
# !pip install openai

Name: openapi
Version: 2.0.0
Summary: Python OpenAPI 2.0 (Swagger) object model
Home-page: https://github.com/globality-corp/openapi
Author: Globality Engineering
Author-email: engineering@globality.com
License: 
Location: /opt/anaconda3/lib/python3.11/site-packages
Requires: inflection, jsonschema
Required-by: 


---
### OPENAI API 호출 예제 

In [2]:
from openai import OpenAI
from pprint import pprint
import os
api_key=os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

client = OpenAI(api_key=api_key)
model = 'gpt-3.5-turbo-1106'

messages = [
    {'role':'system','content': 'You are a helphful assistant.'},
    {'role':'user','content': '군자역 혼잡도 알려줘?'}
]
response = client.chat.completions.create(model=model, messages=messages).model_dump()
pprint(response)

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'content': '죄송합니다, 제가 실시간 정보를 제공할 수 있는 기능은 아직 없어요. '
                                     '혼잡도 정보를 확인하려면 현장에 있는 역 관리소나 공공기관의 웹사이트를 '
                                     '참고하거나, 대중교통 애플리케이션을 이용하는 것이 좋을 것 같아요.',
                          'function_call': None,
                          'role': 'assistant',
                          'tool_calls': None}}],
 'created': 1719973626,
 'id': 'chatcmpl-9gk02ErNKOBnUhNOkisOTVXazpkpP',
 'model': 'gpt-3.5-turbo-1106',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'fp_44132a4de3',
 'usage': {'completion_tokens': 104, 'prompt_tokens': 36, 'total_tokens': 140}}


> gpt 는 json type 으로 답을 넘겨준다. 

---
### 

 * [model dump ]
    : 메서드 체이닝 -> 메서드를 연쇄적으로 호출하는 방식 

---
### ai chatbot 을 만들기 위해서는 프롬프트 엔지니어링이 필요하다.

In [9]:
prompt = """
내일 다음 주제로 미래 교육 포럼에서 발표할 발제문을 작성해야합니다.
발제문의 주요 내용을 불릿 기호로 간략히 정리해 주세요.
주제:{agenda}


""".format(agenda="인공지능 시대의 교육의 역활과 의미")
model = 'gpt-3.5-turbo-1106'
response = client.chat.completions.create(
    model= model,
    messages=[{'role':'user','content':prompt}]
).to_dict
pprint(response)



<bound method BaseModel.to_dict of ChatCompletion(id='chatcmpl-9gk4TZjp8AKGlnr7ciMo6Gzg0LvgY', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='- 인공지능 기술의 발전과 교육의 관련성에 대한 이해\n- 인공지능이 교육 분야에 미치는 영향과 변화\n- 교육의 목표와 방향성을 재조명하는 필요성\n- 인공지능을 효과적으로 활용한 교육 방법과 도구 도입의 중요성\n- 학생들의 능력과 창의성을 개발하는데 인공지능의 활용 방안에 대한 고찰', role='assistant', function_call=None, tool_calls=None))], created=1719973901, model='gpt-3.5-turbo-1106', object='chat.completion', service_tier=None, system_fingerprint='fp_44132a4de3', usage=CompletionUsage(completion_tokens=161, prompt_tokens=97, total_tokens=258))>


In [7]:
print(response)

<bound method BaseModel.model_dump of ChatCompletion(id='chatcmpl-9gk2C7PQ6FnXL2RvlvFR5sibJTl6a', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='- 인공지능이 교육 분야에 미치는 영향\n- 학교 교육과 산업현장의 변화\n- 인공지능을 효과적으로 활용한 교육 방법\n- 교사의 역할과 역량 강화\n- 학생들의 능력과 자기주도적 학습 강화\n- 윤리적 측면과 사회적 영향 고려', role='assistant', function_call=None, tool_calls=None))], created=1719973760, model='gpt-3.5-turbo-1106', object='chat.completion', service_tier=None, system_fingerprint='fp_44132a4de3', usage=CompletionUsage(completion_tokens=137, prompt_tokens=97, total_tokens=234))>
